In [5]:
import tensorflow as tf

tf.__version__

'2.5.0'

In [4]:
def modified_generator_loss(
    discriminator_gen_outputs,
    label_smoothing=0.0,
    weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):
    
    """Modified generator loss for GANs.
    L = -log(sigmoid(D(G(z))))
    This is the trick used in the original paper to avoid vanishing gradients
    early in training. See `Generative Adversarial Nets`
    (https://arxiv.org/abs/1406.2661) for more details.
    
    Args:    
    discriminator_gen_outputs: Discriminator output on generated data. Expected
      to be in the range of (-inf, inf).
      
    label_smoothing: The amount of smoothing for positive labels. This technique
      is taken from `Improved Techniques for Training GANs`
      (https://arxiv.org/abs/1606.03498). `0.0` means no smoothing.
      
    weights: Optional `Tensor` whose rank is either 0, or the same rank as
      `discriminator_gen_outputs`, and must be broadcastable to `labels` (i.e.,
      all dimensions must be either `1`, or the same as the corresponding
      dimension).
      
    scope: The scope for the operations performed in computing the loss.
    
    loss_collection: collection to which this loss will be added.
    
    reduction: A `tf.losses.Reduction` to apply to loss.
    
    add_summaries: Whether or not to add summaries for the loss.
    
    Returns:
    A loss Tensor. The shape depends on `reduction`.
    """
    
    with tf.compat.v1.name_scope(scope, 'generator_modified_loss',
                               [discriminator_gen_outputs]) as scope:
        loss = tf.compat.v1.losses.sigmoid_cross_entropy(
            tf.ones_like(discriminator_gen_outputs), discriminator_gen_outputs,
            weights, label_smoothing, scope, loss_collection, reduction)

        if add_summaries:
            tf.compat.v1.summary.scalar('generator_modified_loss', loss)

    return loss

In [6]:
def modified_discriminator_loss(
    discriminator_real_outputs,
    discriminator_gen_outputs,
    label_smoothing=0.25,
    real_weights=1.0,
    generated_weights=1.0,
    scope=None,
    loss_collection=tf.compat.v1.GraphKeys.LOSSES,
    reduction=tf.compat.v1.losses.Reduction.SUM_BY_NONZERO_WEIGHTS,
    add_summaries=False):
    
    """Same as minimax discriminator loss.
    See `Generative Adversarial Nets` (https://arxiv.org/abs/1406.2661) for more
    details.
    
    Args:
    discriminator_real_outputs: Discriminator output on real data.
    
    discriminator_gen_outputs: Discriminator output on generated data. Expected
      to be in the range of (-inf, inf).
    
    label_smoothing: The amount of smoothing for positive labels. This technique
      is taken from `Improved Techniques for Training GANs`
      (https://arxiv.org/abs/1606.03498). `0.0` means no smoothing.
    
    real_weights: Optional `Tensor` whose rank is either 0, or the same rank as
      `discriminator_gen_outputs`, and must be broadcastable to
      `discriminator_gen_outputs` (i.e., all dimensions must be either `1`, or
      the same as the corresponding dimension).
    
    generated_weights: Same as `real_weights`, but for
      `discriminator_gen_outputs`.
    
    scope: The scope for the operations performed in computing the loss.
    
    loss_collection: collection to which this loss will be added.
    
    reduction: A `tf.losses.Reduction` to apply to loss.
    
    add_summaries: Whether or not to add summaries for the loss.
    
    Returns:
    A loss Tensor. The shape depends on `reduction`.
    """
    
    return minimax_discriminator_loss(
        discriminator_real_outputs,
        discriminator_gen_outputs,
        label_smoothing,
        real_weights,
        generated_weights,
        scope or 'discriminator_modified_loss',
        loss_collection,
        reduction,
        add_summaries)


In [8]:
def test():
    print('success')